In [1]:
import os
import pandas as pd

path = "."  #absolute or relative path to the folder containing the file. 
            #"." for current folder

#read and print values from the dataset to check if it imported correctly
filename_read = os.path.join(path, "pokemon.csv")
df = pd.read_csv(filename_read)
print(df[0:5])
df = df.sample(frac=1).reset_index(drop=True)
print(df[0:5])

   pokedex_number        Name  Type1   Type2  hp  height_m  weight_kg  attack  \
0               1   bulbasaur  Grass  Poison  45       0.7        6.9      49   
1               2     ivysaur  Grass  Poison  60       1.0       13.0      62   
2               3    venusaur  Grass  Poison  80       2.0      100.0     100   
3               4  charmander   Fire     NaN  39       0.6        8.5      52   
4               5  charmeleon   Fire     NaN  58       1.1       19.0      64   

   defense  speed  sp_attack  sp_defense   classfication  generation  \
0       49     45         65          65    Seed Pok�mon           1   
1       63     60         80          80    Seed Pok�mon           1   
2      123     80        122         120    Seed Pok�mon           1   
3       43     65         60          50  Lizard Pok�mon           1   
4       58     80         80          65   Flame Pok�mon           1   

   is_legendary  
0             0  
1             0  
2             0  
3       

In [2]:
#taken from tutorial 3
#df = df.select_dtypes(include=['int', 'float'])

#encode target names to integers
#https://stackoverflow.com/questions/49037286/pandas-string-values-encoding
from sklearn.preprocessing import LabelEncoder
import numpy as np
import save_and_load

print(df['is_legendary'][0:20])

#collect the columns names for non-target features
result = []
for x in df.columns:
    if (x == 'attack') or (x == 'defense') or (x == 'speed') or (x == 'sp_defense') or (x == 'sp_attack') or (x == 'weight_kg') or (x == 'height_m') or (x == 'hp'):
        result.append(x)

#get data (often called X) and target (often calle y) and display its shape
X = df[result].values
yalt = df['is_legendary'].values
print(X.shape)
print(yalt.shape)

from tensorflow import keras

y = keras.utils.to_categorical(df['is_legendary'].to_numpy())

print(y.shape)

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    1
15    0
16    0
17    0
18    1
19    0
Name: is_legendary, dtype: int64
(890, 8)
(890,)
(890, 2)


In [3]:
from sklearn.linear_model import Perceptron
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(X, yalt, test_size=0.15, random_state=4)

#creates a perceptron, with its training parameters
ppn = Perceptron(max_iter=100,tol=0.001,eta0=1)

#trains the model
ppn.fit(X_train,y_train)

#make a predication
y_pred = ppn.predict(X_test)

#evaluates accuracy
print('Accuracy: %.2f' % accuracy_score(y_test, y_pred))

Accuracy: 0.91


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn import metrics

#makes a sequential model and train it using KFolds
model = Sequential()
model.add(Dense(64, input_dim=X.shape[1], activation='relu'))
model.add(Dense(y.shape[1],activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

kf = KFold(6)

for train, test in kf.split(X):
    X_train = X[train]
    y_train = y[train]
    X_test = X[test]
    y_test = y[test]

    model.fit(X_train,y_train,verbose=0,epochs=128)
    pred = model.predict(X_test)
    pred = np.argmax(pred,axis=1)
    y_compare = np.argmax(y_test,axis=1) 
    score = metrics.accuracy_score(y_compare, pred)
    print("Accuracy score: {}".format(score))
    
#questions if you want to save the model
save_and_load.save_model(model, path, "seqKFoldLegendary")

Accuracy score: 0.87248322147651
Accuracy score: 0.912751677852349
Accuracy score: 0.9527027027027027
Accuracy score: 0.9391891891891891
Accuracy score: 0.9391891891891891
Accuracy score: 0.9662162162162162


In [ ]:

pred = model.predict(X) #prediction that uses all the data
print(pred.shape)
print(y.shape)
pred = np.argmax(pred,axis=1)
y_compare = np.argmax(y,axis=1) 
print(pred[0:151])
print(y_compare[0:151])
score = metrics.accuracy_score(y_compare, pred)
print("Accuracy score: {}".format(score))

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

cm = confusion_matrix(np.argmax(y,axis=1), pred)

disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

In [ ]:
print(df['is_legendary'].value_counts())
df2 = pd.read_csv(filename_read)
df2 = df2.sample(frac=1).reset_index(drop=True)
print(df2['is_legendary'].value_counts())